# Conteo de q-Coloraciones en Lattices K×K

Método telescópico con MCMC (Gibbs sampler)

## Librerías

In [5]:
import numpy as np
from numba import jit
import pandas as pd
import time
import os

np.random.seed(42)
os.makedirs('../results', exist_ok=True)

## Funciones

In [6]:
# ============================================================================
# FUNCIONES BÁSICAS
# ============================================================================

def create_lattice_edges(K):
    """Crea aristas de lattice K×K con bordes libres."""
    edges = []
    # Horizontales: (x,y) → (x+1,y)
    for y in range(K):
        for x in range(K - 1):
            edges.append((x, y, x+1, y))
    # Verticales: (x,y) → (x,y+1)
    for y in range(K - 1):
        for x in range(K):
            edges.append((x, y, x, y+1))
    return np.array(edges, dtype=np.int64)


@jit(nopython=True)
def is_valid_coloring(coloring, edges):
    """Verifica si coloración es válida para grafo con aristas 'edges'."""
    for i in range(len(edges)):
        x1, y1, x2, y2 = edges[i]
        if coloring[x1, y1] == coloring[x2, y2]:
            return False
    return True


@jit(nopython=True)
def get_neighbor_colors(x, y, coloring, edges):
    """Obtiene colores de vecinos de (x,y) según aristas en 'edges'."""
    neighbor_colors = set()
    for i in range(len(edges)):
        x1, y1, x2, y2 = edges[i]
        if x1 == x and y1 == y:
            neighbor_colors.add(coloring[x2, y2])
        elif x2 == x and y2 == y:
            neighbor_colors.add(coloring[x1, y1])
    return neighbor_colors


@jit(nopython=True)
def gibbs_step_partial(coloring, edges, q):
    """
    Un paso del Gibbs sampler para grafo parcial.
    Muestrea sobre TODA la lattice K×K pero solo valida aristas en 'edges'.
    """
    K = coloring.shape[0]
    x = np.random.randint(0, K)
    y = np.random.randint(0, K)
    neighbor_colors = get_neighbor_colors(x, y, coloring, edges)
    valid_colors = []
    for c in range(q):
        if c not in neighbor_colors:
            valid_colors.append(c)
    if len(valid_colors) > 0:
        idx = np.random.randint(0, len(valid_colors))
        coloring[x, y] = valid_colors[idx]


@jit(nopython=True)
def run_gibbs_sampler_partial(coloring, edges, q, n_steps):
    """Ejecuta n_steps del Gibbs sampler."""
    for _ in range(n_steps):
        gibbs_step_partial(coloring, edges, q)


def estimate_ratio(K, edges_i_minus_1, edges_i, q, n_samples, n_steps_per_sample, max_steps):
    """
    Estima r_i = Z_i / Z_{i-1}.
    
    Muestrea de G_{i-1} y cuenta cuántas coloraciones son válidas para G_i.
    """
    coloring = np.random.randint(0, q, size=(K, K))
    valid_count = 0
    samples_collected = 0
    steps_executed = 0
    
    for _ in range(n_samples):
        if steps_executed + n_steps_per_sample > max_steps:
            break
        
        run_gibbs_sampler_partial(coloring, edges_i_minus_1, q, n_steps_per_sample)
        steps_executed += n_steps_per_sample
        
        if is_valid_coloring(coloring, edges_i):
            valid_count += 1
        
        samples_collected += 1
    
    ratio = valid_count / samples_collected if samples_collected > 0 else 0.0
    return ratio, samples_collected, steps_executed

In [7]:
# ============================================================================
# FUNCIÓN PRINCIPAL: CONTEO DE q-COLORACIONES
# ============================================================================

def count_colorings(K, q, n_samples, n_steps_per_sample, max_steps_per_ratio):
    """
    Cuenta q-coloraciones en lattice K×K usando método telescópico.
    
    Parámetros:
    -----------
    K : int
        Tamaño de la lattice
    q : int
        Número de colores
    n_samples : int
        Número de muestras por ratio
    n_steps_per_sample : int
        Pasos del Gibbs sampler por muestra
    max_steps_per_ratio : int
        Máximo de pasos totales por ratio
    
    Retorna:
    --------
    dict con:
        - 'K', 'q': parámetros
        - 'log_count': log(Z_{G,q})
        - 'count': Z_{G,q}
        - 'avg_ratio': ratio promedio
        - 'time': tiempo en segundos
    """
    all_edges = create_lattice_edges(K)
    k = len(all_edges)  # k = 2K(K-1) aristas
    N = K * K
    
    # Z_0 = q^(K²)
    log_Z_0 = N * np.log(q)
    
    # Producto telescópico
    log_product = 0.0
    ratios = []
    
    start_time = time.time()
    
    for i in range(1, k + 1):
        edges_i_minus_1 = all_edges[:i-1] if i > 1 else np.array([], dtype=np.int64).reshape(0, 4)
        edges_i = all_edges[:i]
        
        ratio, _, _ = estimate_ratio(
            K, edges_i_minus_1, edges_i, q,
            n_samples, n_steps_per_sample, max_steps_per_ratio
        )
        
        ratio_safe = max(ratio, 1e-300)
        log_product += np.log(ratio_safe)
        ratios.append(ratio)
    
    total_time = time.time() - start_time
    
    log_count = log_Z_0 + log_product
    count = np.exp(log_count) if log_count < 700 else np.inf
    
    return {
        'K': K,
        'q': q,
        'log_count': log_count,
        'count': count,
        'avg_ratio': np.mean(ratios),
        'time': total_time
    }

## Experimentos Completos

In [8]:
# Script para ejecutar múltiples experimentos
def run_experiments(K_range, q_range, n_samples_func, n_steps_func, max_steps_func, output_file):
    """
    Ejecuta experimentos para múltiples (K, q).
    
    n_samples_func: función(K, q) que retorna número de muestras
    n_steps_func: función(K, q) que retorna n_steps_per_sample
    max_steps_func: función(K, q) que retorna max_steps_per_ratio
    """
    results = []
    
    for K in K_range:
        for q in q_range:
            # Calcular parámetros adaptativos
            n_samples = n_samples_func(K, q)
            n_steps = n_steps_func(K, q)
            max_steps = max_steps_func(K, q)
            
            print(f"K={K:2d} q={q:2d} n_samples={n_samples:,} n_steps={n_steps:,} max_steps={max_steps:,} ... ", end='', flush=True)
            
            result = count_colorings(K, q, n_samples, n_steps, max_steps)
            
            print(f"✓ log(Z)={result['log_count']:7.2f} Z≈{result['count']:.2e} t={result['time']:.1f}s")
            
            results.append(result)
            
            # Guardar incremental
            pd.DataFrame(results).to_csv(output_file, index=False)
    
    return pd.DataFrame(results)


# ============================================================================
# PARÁMETROS ADAPTATIVOS (considerando K y q)
# ============================================================================

def get_n_samples(K, q):
    """Número de muestras adaptativo según K y q."""
    # La dificultad aumenta con K grande y q pequeño
    if q == 2:
        if K <= 5:
            return 100_000
        elif K <= 10:
            return 300_000
        else:
            return 500_000
    elif q == 3:
        if K <= 5:
            return 50_000
        elif K <= 10:
            return 150_000
        else:
            return 300_000
    elif q <= 5:
        return 30_000
    else:
        return 10_000


def get_n_steps_per_sample(K, q):
    """Pasos por muestra: aumenta para q pequeño y K grande."""
    base_steps = int(K*K * np.log(K*K + 1) / np.log(q + 1))
    
    # Multiplicador según dificultad
    if q == 2:
        if K <= 5:
            multiplier = 3
        else:
            multiplier = 5
    elif q == 3:
        if K <= 5:
            multiplier = 2
        else:
            multiplier = 4
    else:
        multiplier = 1
    
    return base_steps * multiplier


def get_max_steps(K, q):
    """Máximo de pasos: más para problemas difíciles (K grande, q pequeño)."""
    # Base según K
    if K <= 5:
        base = 30_000_000
    elif K <= 10:
        base = 20_000_000
    elif K <= 15:
        base = 10_000_000
    else:
        base = 5_000_000
    
    # Multiplicador según q
    if q == 2:
        if K <= 5:
            multiplier = 2
        else:
            multiplier = 4
    elif q == 3:
        if K <= 5:
            multiplier = 1.5
        else:
            multiplier = 3
    else:
        multiplier = 1
    
    return int(base * multiplier)

In [9]:
# EJECUTAR experimentos completos
df = run_experiments(
    K_range=range(3, 21),
    q_range=range(2, 12),
    n_samples_func=get_n_samples,
    n_steps_func=get_n_steps_per_sample,
    max_steps_func=get_max_steps,
    output_file='../results/colorings.csv'
)

K= 3 q= 2 n_samples=100,000 n_steps=54 max_steps=60,000,000 ... ✓ log(Z)=-2761.03 Z≈0.00e+00 t=10.3s
K= 3 q= 3 n_samples=50,000 n_steps=28 max_steps=45,000,000 ... ✓ log(Z)=-1375.34 Z≈0.00e+00 t=2.7s
K= 3 q= 4 n_samples=30,000 n_steps=12 max_steps=30,000,000 ... ✓ log(Z)=   9.17 Z≈9.65e+03 t=0.8s
K= 3 q= 5 n_samples=30,000 n_steps=11 max_steps=30,000,000 ... ✓ log(Z)=  11.86 Z≈1.42e+05 t=0.8s
K= 3 q= 6 n_samples=10,000 n_steps=10 max_steps=30,000,000 ... ✓ log(Z)=  13.97 Z≈1.17e+06 t=0.2s
K= 3 q= 7 n_samples=10,000 n_steps=9 max_steps=30,000,000 ... ✓ log(Z)=  15.68 Z≈6.47e+06 t=0.2s
K= 3 q= 8 n_samples=10,000 n_steps=9 max_steps=30,000,000 ... ✓ log(Z)=  17.11 Z≈2.71e+07 t=0.2s
K= 3 q= 9 n_samples=10,000 n_steps=9 max_steps=30,000,000 ... ✓ log(Z)=  18.37 Z≈9.47e+07 t=0.2s
K= 3 q=10 n_samples=10,000 n_steps=8 max_steps=30,000,000 ... ✓ log(Z)=  19.45 Z≈2.81e+08 t=0.2s
K= 3 q=11 n_samples=10,000 n_steps=8 max_steps=30,000,000 ... ✓ log(Z)=  20.47 Z≈7.74e+08 t=0.2s
K= 4 q= 2 n_samples=1

KeyboardInterrupt: 